# KalKalori — BareTubeHeatExchanger Test (with Δp)

Minimal end-to-end validation notebook.

In [12]:
import sys
import os

# Ensure repository root is on sys.path so `core` is importable when running this notebook
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

# Imports — KalKalori BareTubeHeatExchanger test

from core.geometry.tube import BareTube
from core.geometry.bundle import TubeBundle

from core.heat_transfer.internal_flow import (
    FluidProps as TubeFluidProps,
)

from core.heat_transfer.outside_flow import (
    FluidProps as OutsideFluidProps,
)

from core.heat_transfer.streams import (
    SensibleHeatStream,
)

from core.models.bare_tube import (
    BareTubeHeatExchanger,
)



In [13]:
# Geometry

tube = BareTube(
    D_i=0.015,   # m
    D_o=0.018,   # m
    length_total=1.34,      # m
    length_effective=1.33,  # m
)

bundle = TubeBundle(
    tube=tube,
    n_rows=4,
    n_tubes_per_row=20,
    pitch_transverse=0.025,
    pitch_longitudinal=0.025,
    layout="inline",
    n_passes_tube=2,
    flow_arrangement="counterflow", #   "crossflow" / "counterflow" / "cocurrentflow"
)

In [14]:
# Tube-side fluid (water)

tube_props = TubeFluidProps(
    rho=973.0,        # kg/m3
    mu=0.00036,       # Pa*s
    k=0.67,           # W/(m*K)
    cp=4196.0        # J/(kg*K)
)

m_dot_water = 3.0   # kg/s


In [15]:
# Outside forced flow (air)

outside_props = OutsideFluidProps(
    rho=1.16,        # kg/m3
    mu=0.00002,       # Pa*s
    k=0.03,         # W/(m*K)
    cp=1026.4        # J/(kg*K)
)

m_dot_air = 5.5  # kg/s


In [16]:
hot_stream = SensibleHeatStream(
    C=m_dot_water * tube_props.cp,
    T_in=273.15 + 80       # K
)

cold_stream = SensibleHeatStream(
    C=m_dot_air * outside_props.cp,
    T_in=273.15 + 20       # K
)


In [17]:
# Heat exchanger model

hx = BareTubeHeatExchanger(
    bundle=bundle,
    wall_k=16.0       # W/(m*K), steel
)

In [18]:
import inspect
from core.models.bare_tube import BareTubeHeatExchanger
import core.models.bare_tube as bt

print("bare_tube.py loaded from:", bt.__file__)
print("solve signature:", inspect.signature(BareTubeHeatExchanger.solve))


bare_tube.py loaded from: c:\Users\pawel\GitHub\kalkalori\core\models\bare_tube.py
solve signature: (self, hot_stream: 'EnergyStream', cold_stream: 'EnergyStream', *, m_dot_tube_side: 'float', tube_side_props: 'InternalFlowFluidProps', m_dot_outside: 'float | None' = None, outside_props: 'OutsideFlowFluidProps | None' = None, zeta_dp_outside: 'float' = 1.2, K_inlet: 'float' = 0.5, K_outlet: 'float' = 1.0, K_turn: 'float' = 1.5, h_o: 'float | None' = None, flow_arrangement: 'str | None' = None) -> 'HXResult'


In [19]:
result = hx.solve(
    hot_stream=hot_stream,
    cold_stream=cold_stream,
    m_dot_tube_side=m_dot_water,
    tube_side_props=tube_props,
    m_dot_outside=m_dot_air,
    outside_props=outside_props,
)



In [20]:
# Print full results object

print("=== HEAT EXCHANGER RESULTS ===\n")

print("Input streams:")
print(f"  Hot side : T_in = {hot_stream.inlet_temperature() - 273.15:.2f} °C,  C = {hot_stream.capacity_rate():.1f} W/K")
print(f"  Cold side: T_in = {cold_stream.inlet_temperature() - 273.15:.2f} °C,  C = {cold_stream.capacity_rate():.1f} W/K\n")

print("Geometry:")
print(f"  Tube passes       : {hx.bundle.n_passes_tube}")
print(f"  Flow arrangement  : {hx.bundle.flow_arrangement}")
print(f"  Tubes total       : {hx.bundle.n_tubes_total}")
print(f"  Rows              : {hx.bundle.n_rows}")
print(f"  Length effective  : {hx.bundle.tube.length_effective:.2f} m")
print(f"  Length total      : {hx.bundle.tube.length_total:.2f} m\n")

print("Heat transfer areas:")
print(f"  A_i       = {result.A_i:.3f} m^2")
print(f"  A_o       = {result.A_o:.3f} m^2")
print(f"  A_frontal = {result.A_frontal:.3f} m^2\n")

print("Thermal performance:")
print(f"  UA   = {result.UA:.1f} W/K")
print(f"  eps  = {result.eps:.3f}")
print(f"  Q    = {result.Q:.1f} W")
print(f"  T_hot_out  = {result.T_hot_out - 273.15:.2f} °C")
print(f"  T_cold_out = {result.T_cold_out - 273.15:.2f} °C\n")

print("Tube-side (internal):")
print(f"  v   = {result.tube_side_thermal.v:.2f} m/s")
print(f"  Re  = {result.tube_side_thermal.Re:.0f}")
print(f"  h   = {result.tube_side_thermal.h:.1f} W/m^2/K")
print(f"  dp  = {result.tube_side_hydraulic.dp_total:.1f} Pa\n")

print("Outside-side:")
print(f"  v   = {result.outside_side_thermal.v:.2f} m/s")
print(f"  Re  = {result.outside_side_thermal.Re:.0f}")
print(f"  h   = {result.outside_side_thermal.h:.1f} W/m^2/K")
print(f"  dp  = {result.outside_side_hydraulic.dp_total:.1f} Pa")

=== HEAT EXCHANGER RESULTS ===

Input streams:
  Hot side : T_in = 80.00 °C,  C = 12588.0 W/K
  Cold side: T_in = 20.00 °C,  C = 5645.2 W/K

Geometry:
  Tube passes       : 2
  Flow arrangement  : counterflow
  Tubes total       : 80
  Rows              : 4
  Length effective  : 1.33 m
  Length total      : 1.34 m

Heat transfer areas:
  A_i       = 5.014 m^2
  A_o       = 6.017 m^2
  A_frontal = 0.665 m^2

Thermal performance:
  UA   = 455.4 W/K
  eps  = 0.076
  Q    = 25814.0 W
  T_hot_out  = 77.95 °C
  T_cold_out = 24.57 °C

Tube-side (internal):
  v   = 0.44 m/s
  Re  = 17684
  h   = 3703.6 W/m^2/K
  dp  = 724.1 Pa

Outside-side:
  v   = 7.13 m/s
  Re  = 7444
  h   = 78.2 W/m^2/K
  dp  = 141.5 Pa


Expected:
- Re >> 4000 (turbulent water flow)
- Q > 0
- T_hot_out < T_hot_in
- T_cold_out > T_cold_in
- dp > 0